In [1]:
### Uncomment this code if these modules are not installed
# !pip install pandas
# !pip install google-search-results

In [2]:
# Importing libraries
from bs4 import BeautifulSoup
import os
from serpapi import GoogleSearch
import requests
import time
import re
from time import sleep
import json

import pymongo
from pymongo import MongoClient
from pymongo import TEXT

In [3]:
#Function to save html files
def saveString(html, filename):
	try:
		file = open(filename,"w")
		file.write(str(html))
		file.close()
	except Exception as ex:
		print('Error: ' + str(ex))

#Function to load html files
def loadString(f):
	try:
		html = open(f, "r", encoding='utf-8').read()
		return(html)
	except Exception as ex:
		print('Error: ' + str(ex))

#To replicate actual user behaviour on a browser
header_vars = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"}

### Downloading .htm files for job listing pages

In [4]:
seqRange = range(0, 400)

#Creating a sequence for downloading 40 listing pages
pages = []
for i in seqRange:
   if i % 10 == 0:
      pages.append(i)

#Iterating through 40 URLs to download each page
for pageNum in pages:
    URL = "https://www.indeed.com/jobs?q=analyst&start="+ str(pageNum) + "&vjk=ce8b0fe1bba304ca"
    page = requests.get(URL, headers = header_vars)
    
    #Creating a Beautiful Soup Object
    soup = BeautifulSoup(page.content, "html.parser")
    
    #Downloading each page in .htm format
    saveString(soup, "Indeed_"+str(pages.index(pageNum)+1)+".htm")
    
    #Wait after downloading each page
    sleep(5)

### Scraping data from Indeed

In [5]:
#Connect to MongoDB client
client = MongoClient('localhost', 27017)
jobs_db = client['jobs_data']
jobs_Col = jobs_db['jobs_data']


for pageNum in pages:
    
    #Download page in htm format
    htmlFile = loadString("Indeed_"+str(pages.index(pageNum)+1)+".htm")
    soup = BeautifulSoup(htmlFile, 'html.parser')
    tags = soup.select('div[class^="slider_container"]')

    for j in range(len(tags)):
        tag = tags[j]
        jobTitle = tag.find("h2").get_text()
        jobOrg = tag.find("span", {"class" : "companyName"}).get_text()
        jobLocation = tag.find("div", {"class" : "companyLocation"}).get_text()

        try:
            postDate = tag.find("tr", {"class" : "underShelfFooter"}).find('td').find('div').find('span').get_text().split("Posted")[1]
        except:
            postDate = ""
            
        jobType = ""

        try:    
            #Using if-else condition to check if the tag is of type 'Salary' or 'Job type'
            if tag.find("div", {"class" : "metadata"}).find('svg')['aria-label'] == "Salary":
                jobSal = tag.find("div", {"class" : "metadata"}).get_text()
            elif tag.find("div", {"class" : "metadata"}).find('svg')['aria-label'] == "Job type":
                jobType = tag.find("div", {"class" : "metadata"}).get_text()
            else:
                jobSal = ""
                jobType = ""
        except:
            jobSal = ""
            jobType = ""
        
        jobSource = "Indeed"
        
        #Printing job details from Indeed on console
        print("Company Name: ", jobOrg)
        print("Job Title: ", jobTitle)
        print("Location: ", jobLocation)
        print("Salary Range: ", jobSal)
        print("Posted: ", postDate)
        print("Job Type: ", jobType)
        print("\n")

        #Creating an array for keys (which will be part of a dictionary)
        varList = ["CompanyName", "JobTitle", "JobLocation", "SalaryRange", "Posted", "Job Type", "JobSource"]
        
        #Creating an array for values of the above keys
        paramList = [jobOrg, jobTitle, jobLocation, jobSal, postDate, jobType, jobSource]
        
        #Creating  a dictionary of job details
        indeed_sequence = dict(zip(varList, paramList))
#         jobs_Col.drop()

        #Inserting each row in MongoDB
        jobs_Col.insert_one(indeed_sequence)
    

Company Name:  SiriusXM
Job Title:  Music Analyst 1
Location:  San Francisco, CA 94103 (South Of Market area)+7 locations
Salary Range:  
Posted:  30+ days ago
Job Type:  


Company Name:  Invisible Technologies
Job Title:  newOperations Analyst
Location:  Remote in San Francisco, CA 94104
Salary Range:  Up to $60,000 a year
Posted:  1 day ago
Job Type:  


Company Name:  Cox Communications
Job Title:  newData Analyst
Location:  Remote in United States
Salary Range:  $23 - $25 an hour
Posted:  Today
Job Type:  


Company Name:  Google
Job Title:  newSecurity and Abuse Data Analyst, Safe Browsing
Location:  San Francisco, CA+1 location
Salary Range:  $23 - $25 an hour
Posted:  Just posted
Job Type:  Full-time


Company Name:  1Starr
Job Title:  Business Analyst
Location:  Remote
Salary Range:  $72,017 - $97,717 a year
Posted:  
Job Type:  


Company Name:  ZM Financial Systems
Job Title:  newAnalyst
Location:  San Francisco, CA+1 location
Salary Range:  
Posted:  Just posted
Job Type:  

### Scraping data from Google Jobs API

In [6]:
#Creating a sequence for getting data from 10 pages
gRange = range(0, 100)

gpages = []
for i in gRange:
   if i % 10 == 0:
      gpages.append(i)

#Iterating through 10 URLs to get data from each page 
for pageNum in gpages:
  url = "https://serpapi.com/search.json?engine=google_jobs&q=analyst&hl=en&start="+ str(pageNum) +"&api_key=783f3db41babc418337fa011423f23591698d6433f3f04e2acd3ec781c7fd48c"
  
  #Using get request to get json data
  r= requests.get(url)
  data = r.json()
  jobs_results = data['jobs_results']

  
  for i in range(len(jobs_results)):
    
    title = jobs_results[i]['title']
    company = jobs_results[i]['company_name']
    location = jobs_results[i]['location']
    
    try:
        posted = jobs_results[i]['detected_extensions']['posted_at']
    except:
        posted = ""
    
    try:
        jobType = jobs_results[i]['detected_extensions']['schedule_type']
    except:
        jobType = ""
    
    salary = ""
    jobSource = "Google Jobs"
    
    #Printing job details from Google Jobs on console
    print("Title: ", title)
    print("Company: ", company)
    print("Location: ", location)
    print("Posted: ", posted)
    print("JobType: ", jobType)
    print("\n")
    
    #Creating an array for keys (which will be part of a dictionary)
    goog_varList = ["CompanyName", "JobTitle", "JobLocation", "SalaryRange", "Posted", "Job Type", "JobSource"]
    
    #Creating an array for values of the above keys
    goog_paramList = [company, title, location, salary, posted, jobType, jobSource]
    
    #Creating  a dictionary of job details
    google_sequence = dict(zip(goog_varList, goog_paramList))
    
    #Inserting each row in MongoDB
    jobs_Col.insert_one(google_sequence)
    

Title:  Business Analyst
Company:  Staffigo
Location:  Huntington, WV
Posted:  2 days ago
JobType:  Full-time


Title:  Single Family Analyst (hybrid work schedule)
Company:  Freddie Mac
Location:  McLean, VA
Posted:  4 days ago
JobType:  Full-time


Title:  Financial Analyst
Company:  EVMS Consulting Group
Location:  Norfolk, VA
Posted:  4 days ago
JobType:  Full-time


Title:  Senior Financial Analyst
Company:  Altria
Location:  Richmond, VA
Posted:  
JobType:  Full-time


Title:  Program Analyst Associate
Company:  SAIC Career Site
Location:  Alexandria, VA
Posted:  4 days ago
JobType:  Full-time


Title:  Business Systems Analyst 1
Company:  IQVIA
Location:  Columbia, MD
Posted:  15 days ago
JobType:  Full-time


Title:  Entry Level - Data Analyst
Company:  General Motors
Location:  Washington, DC
Posted:  7 days ago
JobType:  Full-time


Title:  BUSINESS ANALYST
Company:  Sentara
Location:  Virginia Beach, VA
Posted:  
JobType:  Full-time


Title:  Business Analyst
Company:  Celer

### Adding text index on Job Title and Job Location

In [7]:
#Connect to MongoDB client
client = MongoClient('localhost', 27017)
jobs_db = client['jobs_data']
jobs_Col = jobs_db['jobs_data']

#Adding text index on Job Title and Job Location
jobs_Col.create_index([('JobTitle', TEXT),('JobLocation',TEXT)])
print(jobs_Col.index_information())

{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'JobTitle_text_JobLocation_text': {'v': 2, 'key': [('_fts', 'text'), ('_ftsx', 1)], 'weights': SON([('JobLocation', 1), ('JobTitle', 1)]), 'default_language': 'english', 'language_override': 'language', 'textIndexVersion': 3}}


### Displaying the collection

In [8]:
query = {}

cursor = jobs_Col.find(query)
try:
	for doc in cursor:
		print(doc)
finally:
	client.close()

{'_id': ObjectId('62398342aa2cae35a24820ae'), 'CompanyName': 'Global Star Design, Inc.', 'JobTitle': 'newCategory Analyst', 'JobLocation': 'Pleasanton, CA 94588', 'SalaryRange': '$65,000 - $75,000 a year', 'Posted': 'Just posted', 'Job Type': '', 'JobSource': 'Indeed'}
{'_id': ObjectId('62398342aa2cae35a24820af'), 'CompanyName': 'Riverbend Lending', 'JobTitle': 'Private Money Loan Production Analyst', 'JobLocation': 'San Francisco, CA 94104 (Financial District/South Beach area)', 'SalaryRange': '$80,000 - $140,000 a year', 'Posted': '30+ days ago', 'Job Type': '', 'JobSource': 'Indeed'}
{'_id': ObjectId('62398342aa2cae35a24820b0'), 'CompanyName': 'TrendUp Now', 'JobTitle': 'Investment Analyst / Trader', 'JobLocation': 'Remote', 'SalaryRange': '$85,000 - $135,000 a year', 'Posted': '', 'Job Type': '', 'JobSource': 'Indeed'}
{'_id': ObjectId('62398342aa2cae35a24820b1'), 'CompanyName': 'Grant Street Group', 'JobTitle': 'newBusiness Analyst – West Coast', 'JobLocation': '+1 locationRemote'